In [1]:
import pandas as pd

In [5]:
import pandas as pd

file_path = "yellow_tripdata_2021-01.parquet"  # Your file path

try:
    df = pd.read_parquet(file_path).head(100)  # Read and select in one line

    styles = [
        {'selector': 'th', 'props': [('border', '1px solid black'), ('padding', '5px')]},
        {'selector': 'td', 'props': [('border', '1px solid black'), ('padding', '5px')]},
        {'selector': 'table', 'props': [('border-collapse', 'collapse')]}
    ]
    
    display(df.style.format(precision=2).set_table_styles(styles))

except FileNotFoundError:
    print(f"File not found at {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2021-01-01 00:30:10,2021-01-01 00:36:12,1.00,2.10,1.00,N,142,43,2,8.00,3.00,0.50,0.00,0.00,0.30,11.80,2.50,nan
1,1,2021-01-01 00:51:20,2021-01-01 00:52:19,1.00,0.20,1.00,N,238,151,2,3.00,0.50,0.50,0.00,0.00,0.30,4.30,0.00,nan
2,1,2021-01-01 00:43:30,2021-01-01 01:11:06,1.00,14.70,1.00,N,132,165,1,42.00,0.50,0.50,8.65,0.00,0.30,51.95,0.00,nan
3,1,2021-01-01 00:15:48,2021-01-01 00:31:01,0.00,10.60,1.00,N,138,132,1,29.00,0.50,0.50,6.05,0.00,0.30,36.35,0.00,nan
4,2,2021-01-01 00:31:49,2021-01-01 00:48:21,1.00,4.94,1.00,N,68,33,1,16.50,0.50,0.50,4.06,0.00,0.30,24.36,2.50,nan
5,1,2021-01-01 00:16:29,2021-01-01 00:24:30,1.00,1.60,1.00,N,224,68,1,8.00,3.00,0.50,2.35,0.00,0.30,14.15,2.50,nan
6,1,2021-01-01 00:00:28,2021-01-01 00:17:28,1.00,4.10,1.00,N,95,157,2,16.00,0.50,0.50,0.00,0.00,0.30,17.30,0.00,nan
7,1,2021-01-01 00:12:29,2021-01-01 00:30:34,1.00,5.70,1.00,N,90,40,2,18.00,3.00,0.50,0.00,0.00,0.30,21.80,2.50,nan
8,1,2021-01-01 00:39:16,2021-01-01 01:00:13,1.00,9.10,1.00,N,97,129,4,27.50,0.50,0.50,0.00,0.00,0.30,28.80,0.00,nan
9,1,2021-01-01 00:26:12,2021-01-01 00:39:46,2.00,2.70,1.00,N,263,142,1,12.00,3.00,0.50,3.15,0.00,0.30,18.95,2.50,nan


In [7]:
import pandas as pd

file_path = "yellow_tripdata_2021-01.parquet"  # Your file path
table_name = "yellow_tripdata"  # Or any name you want

try:
    df = pd.read_parquet(file_path).head(100)
    schema = pd.io.sql.get_schema(df, name=table_name)
    print(schema)
except FileNotFoundError:
    print(f"File not found at {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

CREATE TABLE "yellow_tripdata" (
"VendorID" INTEGER,
  "tpep_pickup_datetime" TIMESTAMP,
  "tpep_dropoff_datetime" TIMESTAMP,
  "passenger_count" REAL,
  "trip_distance" REAL,
  "RatecodeID" REAL,
  "store_and_fwd_flag" TEXT,
  "PULocationID" INTEGER,
  "DOLocationID" INTEGER,
  "payment_type" INTEGER,
  "fare_amount" REAL,
  "extra" REAL,
  "mta_tax" REAL,
  "tip_amount" REAL,
  "tolls_amount" REAL,
  "improvement_surcharge" REAL,
  "total_amount" REAL,
  "congestion_surcharge" REAL,
  "airport_fee" REAL
)


In [13]:
from sqlalchemy import create_engine

In [19]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, DateTime, Boolean

# ... (your code to read the Parquet file into the DataFrame 'df')

try:
    conn_params = {
        "host": "localhost",
        "port": 5432,
        "database": "ny_taxi",
        "user": "root",
        "password": "bmaleh"  # VERIFY THIS PASSWORD!
    }

    engine_str = f"postgresql://{conn_params['user']}:{conn_params['password']}@{conn_params['host']}:{conn_params['port']}/{conn_params['database']}"
    engine = create_engine(engine_str)

    metadata = MetaData()
    table_name = "yellow_taxi_data"

    table = Table(table_name, metadata,
                  Column('VendorID', Integer),
                  Column('tpep_pickup_datetime', DateTime),
                  Column('tpep_dropoff_datetime', DateTime),
                  Column('passenger_count', Integer),
                  Column('trip_distance', Float),
                  Column('RatecodeID', Integer),
                  Column('store_and_fwd_flag', String),
                  Column('PULocationID', Integer),
                  Column('DOLocationID', Integer),
                  Column('payment_type', Integer),
                  Column('fare_amount', Float),
                  Column('extra', Float),
                  Column('mta_tax', Float),
                  Column('tip_amount', Float),
                  Column('tolls_amount', Float),
                  Column('improvement_surcharge', Float),
                  Column('total_amount', Float),
                  Column('congestion_surcharge', Float),
                  Column('airport_fee', Float)
                  )

    metadata.create_all(engine)

    df.to_sql(name=table_name, con=engine, if_exists='replace', index=False, method='multi')

    print(f"Data successfully written to table '{table_name}'")

except FileNotFoundError:
    print(f"File not found at {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

Data successfully written to table 'yellow_taxi_data'


In [23]:
import pandas as pd
from sqlalchemy import create_engine

# ... (your code to read Parquet file into DataFrame 'df')

try:
    conn_params = {"host": "localhost", "port": 5432, "database": "ny_taxi", "user": "root", "password": "bmaleh"}  
    engine_str = f"postgresql://{conn_params['user']}:{conn_params['password']}@{conn_params['host']}:{conn_params['port']}/{conn_params['database']}"
    engine = create_engine(engine_str)

    with engine.connect() as conn:
        df.to_sql("yellow_taxi_data", conn, if_exists='replace', index=False, method='multi')

    print("Data written successfully.")

except Exception as e:
    print(f"Error: {e}")

Data written successfully.


In [19]:
import pandas as pd
import pyarrow.parquet as pq
from sqlalchemy import create_engine

# Database connection details 
conn_params = {"host": "localhost", "port": 5432, "database": "ny_taxi", "user": "root", "password": "bmaleh"}  
engine_str = f"postgresql://{conn_params['user']}:{conn_params['password']}@{conn_params['host']}:{conn_params['port']}/{conn_params['database']}"
engine = create_engine(engine_str)

chunksize = 100000

parquet_file = pq.ParquetFile('yellow_tripdata_2021-01.parquet')

for batch in parquet_file.iter_batches(batch_size=chunksize):
    chunk = batch.to_pandas()
    with engine.connect() as conn:
        chunk.to_sql("yellow_taxi_data", conn, if_exists='append', index=False, method='multi')

print("Data insertion complete.")




Data insertion complete.


In [21]:
import pandas as pd
import pyarrow.parquet as pq
from sqlalchemy import create_engine

# Database connection details (VERIFY PASSWORD!)
conn_params = {"host": "localhost", "port": 5432, "database": "ny_taxi", "user": "root", "password": "bmaleh"}  
engine_str = f"postgresql://{conn_params['user']}:{conn_params['password']}@{conn_params['host']}:{conn_params['port']}/{conn_params['database']}"
engine = create_engine(engine_str)

chunksize = 100000

parquet_file = pq.ParquetFile('yellow_tripdata_2021-01.parquet')

try:
    for batch in parquet_file.iter_batches(batch_size=chunksize):
        chunk = batch.to_pandas()

        # Convert date/time columns
        if 'tpep_pickup_datetime' in chunk.columns:
            chunk['tpep_pickup_datetime'] = pd.to_datetime(chunk['tpep_pickup_datetime'])
        if 'tpep_dropoff_datetime' in chunk.columns:
            chunk['tpep_dropoff_datetime'] = pd.to_datetime(chunk['tpep_dropoff_datetime'])

        with engine.connect() as conn:
            chunk.to_sql("yellow_taxi_data", conn, if_exists='append', index=False, method='multi')

    print("Data insertion complete.")

except Exception as e:
    print(f"An error occurred: {e}")

# Verification (optional)
try:
    with engine.connect() as conn:
        print(pd.read_sql("SELECT * FROM yellow_taxi_data LIMIT 0", conn))
except Exception as e:
    print(f"Error during verification: {e}")

Data insertion complete.
Empty DataFrame
Columns: [VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance, RatecodeID, store_and_fwd_flag, PULocationID, DOLocationID, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, congestion_surcharge, airport_fee]
Index: []


In [25]:
import pandas as pd
import pyarrow.parquet as pq
from sqlalchemy import create_engine
import time

# ... (Database connection details as before) ...

chunksize = 100000

parquet_file = pq.ParquetFile('yellow_tripdata_2021-01.parquet')

total_rows_inserted = 0
start_time = time.time()

try:
    # 1. Create the table (header only) - Most Robust Approach
    table_created = False
    for batch in parquet_file.iter_batches(batch_size=chunksize):
        df_head = batch.to_pandas()  # Convert to pandas DataFrame
        if not df_head.empty:  # Check if DataFrame is empty
            try:
                with engine.connect() as conn:
                    df_head.head(0).to_sql("yellow_taxi_data", conn, if_exists='replace', index=False) # Create table
                table_created = True
                break  # Exit loop after creating the table
            except Exception as e:
                print(f"Error creating table (attempt): {e}")  # Print any errors during table creation
                continue # Try the next batch
    if not table_created:
        raise ValueError("Could not create table: No valid data found in Parquet file after checking multiple batches.")

    chunk_number = 1

    for batch in parquet_file.iter_batches(batch_size=chunksize):
        chunk_start_time = time.time()
        chunk = batch.to_pandas()  # Convert to pandas *before* processing
        chunk_len = len(chunk)

        if chunk.empty:  # Check if chunk is empty (can happen at the end)
            continue  # Skip to the next chunk

        print(f"Processing chunk {chunk_number}: Length = {chunk_len}")

        # 2. Convert date/time columns
        if 'tpep_pickup_datetime' in chunk.columns:
            chunk['tpep_pickup_datetime'] = pd.to_datetime(chunk['tpep_pickup_datetime'])
        if 'tpep_dropoff_datetime' in chunk.columns:
            chunk['tpep_dropoff_datetime'] = pd.to_datetime(chunk['tpep_dropoff_datetime'])

        # 3. Insert data
        with engine.connect() as conn:
            insert_start_time = time.time()
            chunk.to_sql("yellow_taxi_data", conn, if_exists='append', index=False, method='multi')
            insert_end_time = time.time()
            print(f"Chunk {chunk_number} Insertion time: {insert_end_time - insert_start_time:.2f} seconds")

        chunk_end_time = time.time()
        print(f"Total time for chunk {chunk_number}: {chunk_end_time - chunk_start_time:.2f} seconds")

        total_rows_inserted += chunk_len
        chunk_number += 1

    # ... (Rest of the timing and verification code as before)

except Exception as e:
    print(f"An error occurred: {e}")

Processing chunk 1: Length = 100000
Chunk 1 Insertion time: 47.41 seconds
Total time for chunk 1: 47.44 seconds
Processing chunk 2: Length = 100000
Chunk 2 Insertion time: 44.93 seconds
Total time for chunk 2: 44.96 seconds
Processing chunk 3: Length = 100000
Chunk 3 Insertion time: 44.39 seconds
Total time for chunk 3: 44.42 seconds
Processing chunk 4: Length = 100000
Chunk 4 Insertion time: 46.11 seconds
Total time for chunk 4: 46.13 seconds
Processing chunk 5: Length = 100000
Chunk 5 Insertion time: 45.74 seconds
Total time for chunk 5: 45.76 seconds
Processing chunk 6: Length = 100000
Chunk 6 Insertion time: 44.45 seconds
Total time for chunk 6: 44.48 seconds
Processing chunk 7: Length = 100000
Chunk 7 Insertion time: 48.02 seconds
Total time for chunk 7: 48.06 seconds
Processing chunk 8: Length = 100000
Chunk 8 Insertion time: 44.73 seconds
Total time for chunk 8: 44.74 seconds
Processing chunk 9: Length = 100000
Chunk 9 Insertion time: 64.32 seconds
Total time for chunk 9: 64.34 